In [1]:
import pandas as pd
import numpy as np

In [3]:
data = pd.read_csv('receipts_items.csv')
data = data.drop('Id', axis=1)


In [5]:
import re

In [6]:
# чистим названия товаров , удаляем все ненужное 
def clean_names(data):
    new_names = []
    for name in data.Name:
        
        name = re.sub('[/\-#.,):;|@*(!?%$+№]', '', name)
        name = re.sub('(\d+\w+)', '', name)
        name = re.sub('(\d+-\w+)', '', name)
        name = re.sub('(\w+/\w+)', '', name)
        name = re.sub('[\r\n]', '', name)
        name = re.sub('\d+', '', name)
        
        name = re.sub('\s[а-яА-ЯёЁ]{0,3}\s', ' ', name)
        name = re.sub('\s[a-zA-Z]{0,3}\s', ' ', name)
        
        name = re.sub('^[a-zA-Z]{0,2}\s', ' ', name)
        name = re.sub('^[а-яА-ЯёЁ]{0,2}\s', ' ', name)        
        
        name = re.sub('\s[a-zA-Z]{0,2}$', ' ', name)
        name = re.sub('\s[а-яА-ЯёЁ]{0,2}$', ' ', name) 
        
        new_names.append(name.lower())
        
    data['Name'] = new_names

In [7]:
data.groupby('ReceiptId').count() # Всего чеков

,Name,Quantity,Sum,StockKeepingUnitId
ReceiptId,,,,
11,4,4,4,0
12,5,5,5,0
14,3,3,3,0
15,2,2,2,0
16,1,1,1,1
...,...,...,...,...
79200,1,1,1,1
79201,2,2,2,0
79202,1,1,1,1


In [8]:
len(data['Name'].unique()) # Всего уникальных товаров

28585

In [10]:
#Открываем файл с размеченными данными
new_data_no_sort_count_2_new = pd.read_csv('new_data_no_sort_count_2_new.csv', encoding='cp1251', sep=';')
new_data_no_sort_count_2_new.head()

,Name,Count_Name,Type,Unnamed: 3
0,пюре,2318,Тип,NaN
1,девочка,1426,девочка,NaN
2,мальчик,1142,мальчик,NaN
3,фрутоняня,1042,Бренд,NaN
4,каша,990,Тип,NaN


In [9]:
from pymystem3 import Mystem

In [12]:
# Блок лемматизирует слова и переводит их в список. Позиция в списке = номеру строки в нашей базе данных
# Данное изощрение необходимо, чтобы ускорить работу Mystem. Он обработает одну строчку быстрее, чем 1 млн строк. Хотя объем 
# данных будет не изменным

# str(search_words)

search_words = ''

# Мусор который необходимо удалить из строчки
bad_chars = [';', ':', '!', '*', '\\n','\\', '\'', '\"', ',', '\n', "'"] 

for i in range(len(data)-1):
    k   = data.loc[i] # Выделяем 1 строчку 
    search_words += data.loc[i]['Name'] + ' q027dhK ' # Выделяем все продукты в один элемент

#Из строчки удаляем мусор
for j in bad_chars:
    search_words = search_words.replace(j, '') 

m = Mystem() #
lemmas = m.lemmatize(search_words) #  Получаем леммы
new_data_lemmas = ''.join(lemmas).split("q027dhK") # Леммы переводим в строчку
# res1 = text.split(";")

In [13]:
#Наши леммы из первого шага переводим в датафрейм и будем соединять с основной таблицей
df = pd.DataFrame(new_data_lemmas)
df.rename(columns={0: 'new_name'}, inplace=True)
df

,new_name
0,салат ассорти
1,суп из чечевица
2,пармак баклажан
3,пюре
4,S19ERERO016-BB S набор защита из 3-х предмет ...
...,...
299725,подгуз Huggies Elite Soft 4 8-1
299726,пеленка Baby Go 6060 5шт в асс
299727,пеленка Baby Go 4060 5шт в асс
299728,смесь Nestle Alfare молочный 40


In [14]:
#Объединяем с основной таблицей
data1 = data
data1 = data1.merge(df, left_index = True, right_index = True)
data1

,ReceiptId,Name,Quantity,Sum,StockKeepingUnitId,new_name
0,11,"Салат ""Ассорти""",1.0,90.0,NaN,салат ассорти
1,11,Суп из Чечевицы,1.0,20.0,NaN,суп из чечевица
2,11,Пармак Баклажан,1.0,89.0,NaN,пармак баклажан
3,11,Пюре,1.0,0.0,NaN,пюре
4,12,S19ERERO016-BB S Набор защиты из 3-х предметов...,1.0,1119.0,NaN,S19ERERO016-BB S набор защита из 3-х предмет ...
...,...,...,...,...,...,...
299725,79204,Подгуз Huggies Elite Soft 4 8-1,1.0,719.0,52.0,подгуз Huggies Elite Soft 4 8-1
299726,79204,Пеленки Baby Go 60*60 5шт в асс,1.0,75.0,NaN,пеленка Baby Go 6060 5шт в асс
299727,79204,Пеленки Baby Go 40*60 5шт в асс,1.0,39.0,NaN,пеленка Baby Go 4060 5шт в асс
299728,79204,Смесь Nestle Alfare молочная 40,2.0,2450.0,NaN,смесь Nestle Alfare молочный 40


In [57]:
def add_columns1 (row):

    # Перебираем все значения из нашего подготовленного файла со всем словами из чека и ищем пересечения
    # Если есть пересечения, то проверяем название столбца совпадает ли? Если совпадает, то возвращаем продукт
    
    # new_data_no_sort_count_2_new - датафрейм с размеченными данными
    # data1 - таблица с объединенными данными
    
#     for uu in data1['new_name'].iloc[5]:
    for uu in data1['new_name']:
        uu_1 = uu
#         print(uu)
        for u in new_data_no_sort_count_2_new['Name'].sort_values(ascending=False):  
            u_1 = u
#             print(u, '-', uu)

            if u in uu:
#                 print('da') 
                
                qwe = new_data_no_sort_count_2_new.loc[new_data_no_sort_count_2_new['Name'] == 
                                                              u]['Type'].unique()
                qwe = ' '.join(qwe.tolist()).lower()
#                 print(qwe)
                if qwe == 'тип':
#                     print(qwe)
                    return u_1
    return u_1

data1['Type'] = ''
data1['Type'] = data.iloc[0:5].apply(add_columns1, axis=1)
data1.head(15)

,ReceiptId,Name,Quantity,Sum,StockKeepingUnitId,new_name,Type
0,11,"Салат ""Ассорти""",1.0,90.00,NaN,салат ассорти,салат
1,11,Суп из Чечевицы,1.0,20.00,NaN,суп из чечевица,салат
2,11,Пармак Баклажан,1.0,89.00,NaN,пармак баклажан,салат
3,11,Пюре,1.0,0.00,NaN,пюре,салат
4,12,S19ERERO016-BB S Набор защиты из 3-х предметов...,1.0,1119.00,NaN,S19ERERO016-BB S набор защита из 3-х предмет ...,салат
5,12,S19ERERS012-MQ 25-28 Роликовые коньки детские ...,1.0,1819.00,NaN,S19ERERS012-MQ 25-28 роликовый конек детский ...,NaN
6,12,S19ERCRO008-BB M Налокотники Elbow pads черный...,1.0,479.00,NaN,S19ERCRO008-BB M налокотник Elbow pads черный...,NaN
7,12,S19ERCRO010-BB S Наколенники Knee Protection K...,1.0,559.00,NaN,S19ERCRO010-BB S наколенник Knee Protection K...,NaN
8,12,S19ERCRS001-BU 36 Ролик. коньки детск. раздвиж...,1.0,3709.00,NaN,S19ERCRS001-BU 36 ролик. конек детск. раздвиж...,NaN
9,14,Креветки по-азиатски 30 см,1.0,537.95,NaN,креветка по-азиатски 30 см,NaN
